In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
from sklearn.model_selection import GridSearchCV

PREPARAMOS LOS DATOS

In [4]:
lluvia_sl_max = pd.read_excel(r'C:\Users\lab_4\comp metada\Estaciones.xlsx', 1, index_col=None, header=None)

In [5]:
lluvia_sl_max.columns = lluvia_sl_max.iloc[1]

In [6]:
lluvia_sl_max = lluvia_sl_max.drop([1])

In [7]:
lluvia_sl_max.isnull().sum()

1
Año hid                  1
San Luis                21
Miraflores              26
Pozo Sarmiento           7
Pie de Medano           40
Valle Viejo             27
Caimancito              12
Casa de Piedra          44
Potrero del Clavillo     9
Piedra  Grande          44
Los Sosa                11
Padilla                 23
La Maria                45
San Luis Tucuman        28
dtype: int64

In [8]:
#SACAMOS TODAS LAS FILAS QUE TENGAN NAN EN LA COLUMA DE SAN LUIS DE TUCUMAN
lluvia_sl_max = lluvia_sl_max.dropna(subset = ["San Luis Tucuman"])

In [9]:
lluvia_sl_max.isnull().sum()

1
Año hid                  0
San Luis                 7
Miraflores              19
Pozo Sarmiento           0
Pie de Medano           14
Valle Viejo             10
Caimancito               1
Casa de Piedra          16
Potrero del Clavillo     0
Piedra  Grande          16
Los Sosa                 2
Padilla                  0
La Maria                17
San Luis Tucuman         0
dtype: int64

In [10]:
#IMPUTAMOS VALORES A LOS NANS USANDO KNN IMPUTER, DISTANCIA EUCLIDEA EN RELACIÓN A LOS VECINOS
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=4)

In [11]:
lluvia_sl_max_filled = imputer.fit_transform(lluvia_sl_max)

In [12]:
df_filled = pd.DataFrame(lluvia_sl_max_filled)

In [15]:
df_filled.columns = lluvia_sl_max.columns

In [ ]:
sns.pairplot(lluvia_sl_max)
plt.show()

In [18]:
y = np.array(df_filled["San Luis Tucuman"])

In [19]:
x = df_filled.drop(['San Luis Tucuman'], axis=1)

In [55]:
#SEPARAMOS TRAIN Y TEST QUE VAMOS A USAR
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)

In [56]:
scaler = preprocessing.StandardScaler().fit(xtrain)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [57]:
xtrain_scal = scaler.transform(xtrain)

In [58]:
xtest_scal = scaler.transform(xtest)  

KNN PARA PREDECIR LOS VALORES PASADOS DE NUESTRA ESTACION QUE NO TENEMOS

In [59]:
neigh = KNeighborsRegressor(weights = "distance")

In [103]:
parameters_k = np.arange(3,5,7)
parameters_knn = [{'n_neighbors': parameters_k}]

In [104]:
n_folds = 5

In [105]:
regresor_neigh = GridSearchCV(neigh, parameters_knn, refit = True, cv=n_folds, verbose=True)

In [106]:
regresor_neigh.fit(xtrain_scal, ytrain)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='distance'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': array([3])}], pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=True)

In [107]:
regresor_neigh.best_params_

{'n_neighbors': 3}

In [108]:
ypred2 = regresor_neigh.predict(xtest_scal)
ypred2

array([116.06142545,  66.73685767,  68.23967161, 100.4033707 ])

In [109]:
# computo la raiz cuadrada del error cuadratico medio entre ytest (etiqueta real) vs ypred (etiqueta estimada por el modelo)
np.sqrt(mean_squared_error(ytest, ypred2))

31.76408661284596

Support vector regresion PARA PREDECIR LOS VALORES DE LA ESTACION QUE NO TENEMOS, LOS PASADOS

In [110]:
svreg = SVR()

In [111]:
n_folds = 5

In [112]:
parameters_svr_lin = [{'kernel':['linear'], 'C': [1, 100]}]

parameters_svr_rbf = [{'kernel':['rbf'] , 'C': [1,100],'gamma': [0.1,0.5] }]

In [113]:
regresor_svr = GridSearchCV(svreg, parameters_svr_lin, cv=n_folds, verbose=True)

In [114]:
# entreno modelo regresor con mi set de entrenamiento (tanto Xtrain como Ytrain)
regresor_svr.fit(xtrain_scal, ytrain.ravel())
#svr_lin.fit(xtrain_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 100], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [115]:
regresor_svr.best_params_

{'C': 1, 'kernel': 'linear'}

In [116]:
regresor_svr.best_score_

-3.8119395738463724

In [117]:
ypred3 = regresor_svr.predict(xtest_scal)
ypred3

array([90.32844201, 89.96732874, 60.19436443, 95.36573387])

In [118]:
np.sqrt(mean_squared_error(ytest, ypred3))

26.138377377867492

ahora uso el modelo para predecir valores previos a los que tenemos de san luis, vamos a usar el SVR

In [289]:
lluvia_sl = pd.read_excel(r'C:\Users\lab_4\comp metada\Estaciones.xlsx', 1, index_col=None, header=None)

In [290]:
lluvia_sl.columns = lluvia_sl.iloc[1]

In [293]:
lluvia_sl = lluvia_sl.drop([0])

In [294]:
lluvia_sl

1,Año hid,San Luis,Miraflores,Pozo Sarmiento,Pie de Medano,Valle Viejo,Caimancito,Casa de Piedra,Potrero del Clavillo,Piedra Grande,Los Sosa,Padilla,La Maria,San Luis Tucuman
2,1946,NaN,NaN,133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1947,NaN,NaN,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1948,NaN,NaN,76.3,NaN,NaN,41.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1949,NaN,NaN,64.4,NaN,NaN,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1950,NaN,NaN,60,NaN,NaN,134,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1951,NaN,NaN,61.2,NaN,NaN,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1952,NaN,88,90,NaN,NaN,108.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1953,58.5,111.5,112,NaN,NaN,133.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1954,90.5,46,96.3,NaN,NaN,107.5,NaN,87.1,NaN,142,NaN,NaN,NaN
11,1955,64.2,65.5,113.5,NaN,NaN,132.5,NaN,55,NaN,115,NaN,NaN,NaN


VOLVEMOS A IMPUTAR VALORES A NANS, VAMOS A SACAR LA COLUMNA DE SAN LUIS Y VAMOS A RECORTAR EL DF PARA QUE SE USEN LAS PRIMERAS 27 FILAS PARA PREDECIR LOS 27 VALORES QUE NECESITAMOS DE SAN LUIS.

In [295]:
lluvia_sl_filled = imputer.fit_transform(lluvia_sl)

In [298]:
df_filled2 = pd.DataFrame(lluvia_sl_filled)

In [300]:
df_filled2.columns = lluvia_sl.columns

In [301]:
df_filled2 = df_filled2.drop(['San Luis Tucuman'], axis = 1)

In [303]:
df_filled2 = df_filled2.iloc[:27]

In [304]:
df_filled2.shape

(27, 13)

In [305]:
x_pasado = scaler.transform(df_filled2)

In [306]:
ypred_pasado =  regresor_svr.predict(x_pasado)

In [307]:
ypred_pasado.shape

(27,)

In [313]:
df_filled2['San Luis Tucuman'] = ypred_pasado

AGREGO LOS VALORES Y TENGO UN NUEVO DATAFRAME QUE TIENE LOS VALORES PREDICHOS CONCATENADO A LOS VALORES QUE YA NOS DIERON

In [367]:
san_tucuman_entero = san_tucuman_entero.to_csv('san_luis_entero.csv')

In [374]:
san_tucuman_entero = pd.read_csv(r'C:\Users\lab_4\comp metada\san_luis_entero.csv')

In [376]:
san_tucuman_entero['Año hid'] = san_tucuman_entero['Año hid'].astype('int')

In [377]:
san_tucuman_entero.set_index('Año hid', inplace = True)

In [378]:
#SEPARAMOS EN TRAIN Y TEST
lluvia_train = san_tucuman_entero['San Luis Tucuman'].iloc[:50]
lluvia_test = san_tucuman_entero['San Luis Tucuman'].iloc[50:]

In [379]:
#MODIFICAMOS LOS INDICES A DATETIME
lluvia_train.index = pd.to_datetime(lluvia_train.index, format='%Y')
lluvia_test.index = pd.to_datetime(lluvia_test.index, format='%Y')

In [380]:
lluvia_train = lluvia_train.astype('float64')
lluvia_test = lluvia_test.astype('float64')

VOLVEMOS A ANALIZAR EL ARIMA PERO AHORA CON TODA LA SECUENCIA NUEVA

In [381]:
#USAMOS CODIGO DE INTERNET, TIENE UNA PARTE PARA EVALUAR EL MODELO DE ARIMA, ES DECIR LOS PARÁMETROS,
# https://machinelearningmastery.com/make-sample-forecasts-arima-python/
# https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/

import warnings
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
 

# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(san_tucuman_entero['San Luis Tucuman'].values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=1283.007
ARIMA(0, 0, 1) MSE=1338.383
ARIMA(0, 0, 2) MSE=1348.417
ARIMA(0, 1, 0) MSE=2345.909
ARIMA(0, 1, 1) MSE=1417.145
ARIMA(0, 1, 2) MSE=1363.861
ARIMA(0, 2, 0) MSE=6874.329
ARIMA(0, 2, 1) MSE=2458.376
ARIMA(0, 2, 2) MSE=1566.608
ARIMA(1, 0, 0) MSE=1337.247
ARIMA(1, 0, 1) MSE=1397.382
ARIMA(1, 0, 2) MSE=1390.483
ARIMA(1, 1, 0) MSE=1924.414
ARIMA(1, 1, 1) MSE=1375.287
ARIMA(1, 1, 2) MSE=1469.813
ARIMA(1, 2, 0) MSE=3970.039
ARIMA(2, 0, 0) MSE=1365.922
ARIMA(2, 0, 1) MSE=1394.266
ARIMA(2, 0, 2) MSE=1412.959
ARIMA(2, 1, 0) MSE=1830.096
ARIMA(2, 1, 1) MSE=1409.706
ARIMA(2, 1, 2) MSE=1413.217
ARIMA(2, 2, 0) MSE=3665.027
ARIMA(2, 2, 1) MSE=1946.944
ARIMA(2, 2, 2) MSE=1579.386
ARIMA(4, 0, 0) MSE=1514.218
ARIMA(4, 0, 1) MSE=1644.311
ARIMA(4, 1, 0) MSE=1640.718
ARIMA(4, 1, 1) MSE=1560.011
ARIMA(4, 2, 0) MSE=2678.529
ARIMA(4, 2, 1) MSE=1774.729
ARIMA(6, 0, 0) MSE=1679.205
ARIMA(6, 0, 1) MSE=1765.800
ARIMA(6, 1, 0) MSE=1777.290
ARIMA(6, 1, 1) MSE=1729.531
ARIMA(6, 2, 0) MSE=2

In [382]:
# FITEAMOS EL MODELO, QUE NOS DA QUE ES UN MODELO DE RUIDO BLANCO (0,0,0)
model = ARIMA(np.asarray(lluvia_train), order=(0,0,0))
model_fit = model.fit(disp=0)
# print summary of fit model
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   50
Model:                     ARMA(0, 0)   Log Likelihood                -251.859
Method:                           css   S.D. of innovations             37.272
Date:                Thu, 27 Feb 2020   AIC                            507.717
Time:                        20:46:18   BIC                            511.541
Sample:                             0   HQIC                           509.174
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         98.2400      5.271     18.638      0.000      87.909     108.571


In [387]:
forecast = model_fit.forecast(steps=10)[0]

In [388]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(lluvia_test, forecast)

22.696000134421713